# 🚀 RAGScore - 1-Minute RAG Audit

Audit your RAG system in one line. Get instant visualizations.

**Features:**
- ✅ Works in Google Colab and Jupyter
- ✅ Supports local LLMs (Ollama) and cloud APIs
- ✅ Rich Object returns: `.df`, `.plot()`, `.corrections`
- ✅ Pass HTTP endpoint or Python function

## 1. Install RAGScore

In [ ]:
!pip install -q ragscore[notebook]

# Safety net: Explicit asyncio patch for Colab
import nest_asyncio

nest_asyncio.apply()
print("✅ RAGScore installed with notebook support")

## 2. Setup LLM

### Option A: Cloud LLM (OpenAI, etc.)

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key

### Option B: Local LLM (Ollama) - FREE & Private!

Run this cell to set up Ollama in Colab (~2 minutes):

In [ ]:
# Install and start Ollama
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
import time

subprocess.Popen("ollama serve", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

!ollama pull llama3
print("✅ Ollama ready!")

## 2.5 Setup: Download Sample Document

This ensures the demo works immediately without manual file uploads.

In [ ]:
# 📥 Download a sample PDF so the demo works out-of-the-box
!mkdir -p docs
!echo "Sample RAG document for testing." > docs/sample.txt
print("✅ Sample document created in docs/")

---

## 3. The "1-Minute Audit" ⚡

This is the money shot. Copy this block to audit any RAG.

In [ ]:
from ragscore import quick_test

# 1. Audit your RAG in one line
result = quick_test(
    endpoint="http://localhost:8000/query",  # Your RAG API
    docs="docs/",                            # Your documents
    n=10,                                    # Number of test questions
)

# 2. See the report
result.plot()

# 3. Inspect failures
bad_rows = result.df[result.df['score'] < 3]
display(bad_rows[['question', 'rag_answer', 'reason']])

---

## 4. Test with a Python Function (No Server!)

Perfect for testing RAG built directly in Colab:

In [ ]:
# Your RAG function
def my_rag(question: str) -> str:
    # Replace with your actual RAG logic
    # e.g., vectorstore.similarity_search(question)
    return "This is a placeholder answer."

# Test it!
result = quick_test(
    endpoint=my_rag,  # Pass function directly
    docs="docs/",
    n=5,
)

result.plot()

---

## 5. Rich Object API

The `QuickTestResult` object gives you everything:

In [ ]:
# Metrics
print(f"Accuracy: {result.accuracy:.1%}")
print(f"Passed: {result.passed}")
print(f"Average Score: {result.avg_score:.1f}/5.0")

# DataFrame for analysis
result.df.head()

In [ ]:
# Corrections for RAG improvement
print(f"{len(result.corrections)} corrections needed")
for c in result.corrections[:3]:
    print(f"\nQ: {c['question'][:60]}...")
    print(f"Wrong: {c['incorrect_answer'][:60]}...")
    print(f"Correct: {c['correct_answer'][:60]}...")

In [ ]:
# Export corrections to file
from ragscore.quick_test import export_corrections

export_corrections(result, "corrections.jsonl")
print("✅ Corrections saved! Inject into your RAG to improve.")

---

## 6. Use in pytest (CI/CD)

```python
# test_rag.py
from ragscore import quick_test

def test_rag_accuracy():
    result = quick_test(
        endpoint="http://localhost:8000/query",
        docs="docs/",
        n=20,
        threshold=0.8,
        silent=True,
    )
    assert result.passed, f"RAG accuracy too low: {result.accuracy:.0%}"
```

---

## 📚 Resources

- **GitHub**: https://github.com/HZYAI/RagScore
- **PyPI**: https://pypi.org/project/ragscore/

⭐ Star us on GitHub if you find this useful!